In [1]:
!pip install pytesseract easyocr opencv-python numpy pandas scikit-learn tensorflow keras

In [3]:
import cv2
import pytesseract
import numpy as np
import pandas as pd
import re
import string
from PIL import Image


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC

In [7]:
df = pd.read_csv("final_data.csv")  # Contains 'Description' and 'Genre' columns
df.drop(columns=['isbn13','isbn10','thumbnail','subtitle','average_rating','published_year','num_pages','ratings_count'], inplace=True) 
df

,title,authors,genre,description
0,Gilead,Marilynne Robinson,Fiction,A NOVEL THAT READERS and critics have been eag...
1,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...
2,The One Tree,Stephen R. Donaldson,Fiction,Volume Two of Stephen Donaldson's acclaimed se...
3,Rage of angels,Sidney Sheldon,Fiction,"A memorable, mesmerizing heroine Jennifer -- b..."
4,The Four Loves,Clive Staples Lewis,Christian,Lewis' work on the nature of love divides love...
...,...,...,...,...
4971,Good Fight,Judd Winick;Matthew Clark,Juvenile Fiction,Presents the adventures of a new group of hero...
4972,"Cry, the Peacock",Anita Desai,Fiction,This book is the story of a young girl obsesse...
4973,Journey to the East,Hermann Hesse,Adventure,This book tells the tale of a man who goes on ...
4974,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,"Wisdom to Create a Life of Passion, Purpose, a..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4976 entries, 0 to 4975
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        4976 non-null   object
 1   authors      4972 non-null   object
 2   genre        4976 non-null   object
 3   description  4976 non-null   object
dtypes: object(4)
memory usage: 155.6+ KB


In [6]:
df['authors'].fillna(method='ffill', inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14556\4203095110.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['authors'].fillna(method='ffill', inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14556\4203095110.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['authors'].fillna(method='ffill', inplace=True)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4976 entries, 0 to 4975
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        4976 non-null   object
 1   authors      4976 non-null   object
 2   genre        4976 non-null   object
 3   description  4976 non-null   object
dtypes: object(4)
memory usage: 155.6+ KB


In [8]:
df['genre'].fillna(method='ffill', inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14556\3190136022.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['genre'].fillna(method='ffill', inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14556\3190136022.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['genre'].fillna(method='ffill', inplace=True)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4976 entries, 0 to 4975
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        4976 non-null   object
 1   authors      4976 non-null   object
 2   genre        4976 non-null   object
 3   description  4976 non-null   object
dtypes: object(4)
memory usage: 155.6+ KB


In [10]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['genre_Label'] = encoder.fit_transform(df['genre'])
genre_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))  # Store genre mapping
print(genre_mapping)

{' Arts ': 0, ' Crime': 1, ' Fiction': 2, 'Acting': 3, 'Adventure ': 4, 'American fiction': 5, 'Arabic fiction': 6, 'Australian fiction': 7, 'Biography & Autobiography': 8, 'Christian ': 9, 'Classical fiction': 10, 'Comics & Graphic Novels': 11, 'Cooking': 12, 'Detective and mystery stories': 13, 'Drama': 14, 'Education': 15, 'Family & Relationships': 16, 'Fantasy ': 17, 'Fiction': 18, 'Friendship': 19, 'Friendship in adolescence': 20, 'Health & Fitness': 21, 'Historical fiction': 22, 'Horror': 23, 'Humor': 24, 'Humorous fiction': 25, 'Inventions': 26, 'JUVENILE FICTION': 27, 'Juvenile Fiction': 28, 'Juvenile Nonfiction': 29, 'Literature': 30, 'Music': 31, 'Nature': 32, 'Philosophy': 33, 'Poetry': 34, 'Political fiction': 35, 'Psychology': 36, 'Romance': 37, 'Science ': 38, 'Technology & Engineering': 39, 'Travel': 40, 'War': 41}


In [11]:
df

,title,authors,genre,description,genre_Label
0,Gilead,Marilynne Robinson,Fiction,A NOVEL THAT READERS and critics have been eag...,18
1,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,13
2,The One Tree,Stephen R. Donaldson,Fiction,Volume Two of Stephen Donaldson's acclaimed se...,18
3,Rage of angels,Sidney Sheldon,Fiction,"A memorable, mesmerizing heroine Jennifer -- b...",18
4,The Four Loves,Clive Staples Lewis,Christian,Lewis' work on the nature of love divides love...,9
...,...,...,...,...,...
4971,Good Fight,Judd Winick;Matthew Clark,Juvenile Fiction,Presents the adventures of a new group of hero...,28
4972,"Cry, the Peacock",Anita Desai,Fiction,This book is the story of a young girl obsesse...,18
4973,Journey to the East,Hermann Hesse,Adventure,This book tells the tale of a man who goes on ...,4
4974,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,"Wisdom to Create a Life of Passion, Purpose, a...",21


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('all')

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(words)

df['Cleaned_Description'] = df['description'].apply(preprocess_text)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['Cleaned_Description']).toarray()
y = df['genre_Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Train the Naïve Bayes model
model = MultinomialNB()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Print accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Ensure target_names matches only the classes present in y_test
unique_labels = np.unique(y_test)  # Get unique labels from y_test
filtered_target_names = [encoder.classes_[i] for i in unique_labels]  # Match class names

# Generate classification report
print(classification_report(y_test, y_pred, labels=unique_labels, target_names=filtered_target_names))


In [ ]:
from sklearn.linear_model import LogisticRegression  # Import the missing module
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Train Logistic Regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

# Predict on the test set
y_pred_lr = lr_model.predict(X_test)

# Print accuracy
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))

# Ensure correct labels for classification report
unique_labels = np.unique(y_test)
print(classification_report(y_test, y_pred_lr, labels=unique_labels))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Initialize and train the Decision Tree model
dt_model = DecisionTreeClassifier(criterion='gini', max_depth=None, random_state=42)
dt_model.fit(X_train, y_train)

# Predict on the test set
y_pred_dt = dt_model.predict(X_test)

# Print accuracy
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))

# Ensure correct labels for classification report
print(classification_report(y_test, y_pred_dt, labels=np.unique(y_test)))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf_model.predict(X_test)

# Print accuracy
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

# Ensure correct labels for classification report
print(classification_report(y_test, y_pred_rf, labels=np.unique(y_test)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Define the parameter grid
param_dist = {
    'n_estimators': [50, 100, 200, 300, 500],  # Number of trees
    'max_depth': [None, 10, 20, 30, 40],       # Maximum depth of trees
    'min_samples_split': [2, 5, 10],           # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],             # Minimum samples required at a leaf node
    'criterion': ['gini', 'entropy'],          # Splitting criteria
    'bootstrap': [True, False]                 # Whether to use bootstrap sampling
}

# Initialize Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Use RandomizedSearchCV to find the best hyperparameters
random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_dist,
    n_iter=20,  # Number of different combinations to try
    cv=5,       # 5-fold cross-validation
    scoring='accuracy',
    verbose=2,
    n_jobs=-1,  # Use all available processors
    random_state=42
)

# Fit the model
random_search.fit(X_train, y_train)

# Get the best parameters
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Train a new model with the best parameters
best_rf_model = RandomForestClassifier(**best_params, random_state=42)
best_rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = best_rf_model.predict(X_test)

# Print accuracy and classification report
print("Optimized Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, labels=np.unique(y_test)))



In [ ]:
!pip install pytesseract
!pip install pillow
!pip install scikit-learn


In [ ]:
import pytesseract
from PIL import Image
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
pip install easyocr

In [ ]:
import easyocr
from PIL import Image

# Create an EasyOCR reader
reader = easyocr.Reader(['en'])  # 'en' stands for English. You can add other languages as needed

# Load the image
image_path = r"C:\Users\ASUS\Desktop\image\lo.jpg"
image = Image.open(image_path)

# Perform OCR and extract text
result = reader.readtext(image_path)

# Print the extracted text
for detection in result:
    print(detection[1])  # The second element contains the recognized text


In [ ]:
def extract_text_easyocr(image_path):
    reader = easyocr.Reader(['en'])  # Initialize EasyOCR reader for English
    results = reader.readtext(image_path, detail=0)  # Extract text from image
    return " ".join(results)  # Return extracted text as a single string

def predict_genre(image_path):
    text = extract_text_easyocr(image_path)
    processed_text = preprocess_text(text)

    # Machine Learning Prediction
    text_tfidf = vectorizer.transform([processed_text]).toarray()
    ml_prediction = rf_model.predict(text_tfidf)
    ml_genre = encoder.inverse_transform(ml_prediction)[0]
    
    return {"Genre": ml_genre}

# Test with a new image
image_path = r"C:\Users\ASUS\Desktop\image\lo.jpg"
prediction = predict_genre(image_path)
print(prediction)
